1. Structured Outputs

2. Amulti-Agent worflow

In [ ]:
from typing import Annotated, TypedDict, List, Dict, Any, Optional
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser

from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
from IPython.display import Image, display
import gradio as gr
import uuid
from dotenv import load_dotenv


d:\Jeevan\Agentic_AI\LangGraph_Langchain\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(override=True)


True

First define the STRUCTURED OUTPUT

In [ ]:
class EvaluatorOutput(BaseModel):
    feedback: str = Field(description="Feedback on the workers response")
    success_criteria_met: bool = Field(description= "Whether the success criteria has been met")
    user_input_needed: bool = Field(description="True if more input is needed from the user, or clarifications, or the assistant is stuck")

And for the State, we'll use TypedDict again

But now we have some real info. to maintain!

The messages uses the reducer. The others are simply values that we overwrite with any state change

In [ ]:
# The state
from langgraph.graph.message import add_messages
class State(TypedDict):
    messages: Annotated[List[Any], add_messages]
    success_criteria: str
    feedback_on_work: Optional[str]
    success_criteria_met: bool
    user_input_needed: bool


In [ ]:
# get our Async Playwright tools

import nest_asyncio
nest_asyncio.apply()
async_browser = create_async_playwright_browser(headless=False) # headful mode
toolkit = PlayWrightBrowserToolkit(async_browser=async_browser)
tools = toolkit.get_tools()

Task exception was never retrieved
future: <Task finished name='Task-2' coro=<Connection.run() done, defined at d:\Jeevan\Agentic_AI\LangGraph_Langchain\.venv\Lib\site-packages\playwright\_impl\_connection.py:303> exception=NotImplementedError()>
Traceback (most recent call last):
  File "C:\Users\DELL\anaconda3\Lib\asyncio\tasks.py", line 314, in __step_run_and_handle_result
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "d:\Jeevan\Agentic_AI\LangGraph_Langchain\.venv\Lib\site-packages\playwright\_impl\_connection.py", line 310, in run
    await self._transport.connect()
  File "d:\Jeevan\Agentic_AI\LangGraph_Langchain\.venv\Lib\site-packages\playwright\_impl\_transport.py", line 133, in connect
    raise exc
  File "d:\Jeevan\Agentic_AI\LangGraph_Langchain\.venv\Lib\site-packages\playwright\_impl\_transport.py", line 120, in connect
    self._proc = await asyncio.create_subprocess_exec(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\DELL\ana

NotImplementedError: 

In [7]:
# Initialize the LLM

worker_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
worker_llm_with_tools = worker_llm.bind_tools(tools)


evaluator_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
evaluator_llm_with_structured_output = evaluator_llm.with_structured_output(EvaluatorOutput)


NameError: name 'tools' is not defined

In [ ]:
# The worker node

def worker(state: State) -> Dict[str, Any]:
    system_message= f"""you are a helpful assistant that can use tools to complete tasks
    You keep working on a task until either you have a question or clarification for the user, or the success criteria is met
    This is the success criteria: {state["success_criteria"]}
    you should reply either with a question for the user about this assignment, or with your final response.
    If you have a question for the user, you need to reply by clearly stating that you are asking a question, and then ask the question.
    
    An example might be:

    Question: Please clarify whether you want a summary or a detailed answer

    If you have finished, reply with the final answer, and don't ask a question. simply reply with the answer.
"""

    if state.get("feedback_on_work"):
        system_message += f"""
        Previously you thought you completed the assignment, but your reply was rejected because the success criteria was not met.
        Here is the feedback of why this was rejected: {state["feedback_on_work"]}
        With this feedback in mind, please continue the assignment, ensuring that you meet the success criteria or ask for more information if needed.
        """

        # Add in the system message to the messages

        found_system_message = False
        messages = state["messages"]
        for message in messages:
            if isinstance(message, SystemMessage):
                message.content = system_message
                found_system_message = True
        
        if not found_system_message:
            messages = [SystemMessage(content=system_message)] + messages

        # Invoke the LLM with tools
        response = [SystemMessage(content=system_message)] + messages

        # return the updated state
        return {"messages": [response],
        }